# Running SNID

For parameters information, check https://people.lam.fr/blondin.stephane/software/snid/howto.html

In [27]:
import os
import glob
import numpy as np
import pandas as pd

directories = glob.glob('spectra/*')

In [ ]:
sn_files

In [40]:
sn_files = [file for file in all_directory_files
                if file.endswith('.fits') is False and 'snid' not in file]
sn_files

['spectra/2023gps/SN2023gps_2023-04-24_00-00-00_NOT_ALFOSC_TNS.txt',
 'spectra/2023gps/phases.txt']

In [75]:
for directory in directories:
    # get every single file for this SN
    all_directory_files = glob.glob(os.path.join(directory, '*')) 

    # get only the text files, excluding SNID outputs from previous runs
    sn_files = [file for file in all_directory_files
                if file.endswith('.fits') is False and 
                'snid' not in file and 
                file.endswith('phases.txt') is False]

    # create a new file in the format that SNID likes
    for file in sn_files:
        sn_df = pd.read_csv(file)
        snid_df = sn_df[['wave', 'flux']]
        outfile = os.path.splitext(file)[0] + '.snid'
        snid_df.to_csv(outfile, header=False, index=False, sep='\t')
        
    # get the files what we just created
    snid_files = [file for file in all_directory_files if file.endswith('.snid')] 
            
    for file in snid_files:
        # go to the SN directory, run SNID and come back
        basename = os.path.basename(file)
        os.chdir(directory)
        !snid inter=0 plot=0 aband=0 usetype=Ia-norm xcorout=0 {basename}
        !pwd
        os.chdir('../..')

        """
        # get the output files and the templates phases
        snidxcor_files = [file for file in all_directory_files 
                          if file.endswith('snidxcor_trim.dat')]
        snidxcor_files.sort()  # important to sort the templates by ranking
        phases = []
        for cor_file in snidxcor_files:
            with open(cor_file) as f:
                first_line = f.readlines()[0]
                phase = first_line.split()[-1].split(')')[0]
                phases.append(phase)
           
        outfile = os.path.join(directory, 'phases.txt')
        np.savetxt(outfile, np.array(phases).T, fmt='%s')
        """
        
        # get SNID output file
        snid_output = [file for file in all_directory_files 
                       if basename.split('.')[0] in file and 
                       file.endswith('snid.output')][0]
        with open(snid_output) as f:
            skiprows = 0
            for i, line in enumerate(f.readlines()):   
                if  line.startswith('#no. sn type'):
                    break
                skiprows += 1

        output_df = pd.read_csv(snid_output, skiprows=skiprows, nrows=999, delim_whitespace=True)
        output_df.to_csv('phases.txt', index=False)

 S u p e r N o v a    I D e n t i f i c a t i o n (SNID v5.0, 24 Aug 2007)
  
 Searching in redshift range: -0.010  1.200 ; rlapmin =  5.00
 Restricting to age range:  -90.0 1000.0
 Restricting to delta range:  -10.0   99.9
 Forcing use of spectral types: Ia-norm
 Reading data file: SN2023gps_2023-04-24_00-00-00_NOT_ALFOSC_TNS.snid
 Restricting to wavelength range:    3800.0   9000.0
 Reading template files...sn00cn sn00cp sn00cu sn00cw sn00dg sn00dk sn00dm sn00dn sn00fa sn01bg sn01dw sn01en sn01ep sn02bg sn02bo sn02bo sn02bz sn02cr sn02cs sn02ef sn02el sn02er sn02er sn02eu sn02fk sn02ha sn02hd sn02he sn02jy sn02kf sn03ai sn03cq sn03he sn03iv sn03U sn04as sn04bg sn04bl sn04bw sn04dt sn04ef sn04eo sn04ey sn04fu sn04fz sn04gs sn05am sn05bc sn05be sn05cf sn05de sn05dv sn05el sn05kc sn05ki sn05ms sn06ac sn06bq sn06bu sn06cf sn06cp sn06cq sn06dm sn06dw sn06ef sn06ej sn06et sn06ev sn06kf sn06lf sn06N sn06or sn06os sn06sr sn07A sn07af sn07bc sn07bd sn07bm sn07fb sn07fr sn07fs sn07gk sn07kk sn

In [76]:
outfile

'spectra/2023gps/SN2023gps_2023-04-24_00-00-00_NOT_ALFOSC_TNS.snid'

In [77]:
output_df

,#no.,sn,type,lap,rlap,z,zerr,age,age_flag,grade
0,1,sn05cf,Ia-norm,0.8597,10.08,0.0287,0.0072,-10.94,0.0,good
1,2,sn02bo,Ia-norm,0.6918,9.17,0.0216,0.0071,-1.50,0.0,good
2,3,sn02bo,Ia-norm,0.6877,8.66,0.0238,0.0075,-3.40,0.0,good
3,4,sn95D,Ia-norm,0.6322,8.42,0.0119,0.0071,8.10,0.0,good
4,5,sn02bo,Ia-norm,0.6918,8.18,0.0352,0.0084,-12.30,0.0,good
...,...,...,...,...,...,...,...,...,...,...
474,474,sn89B,Ia-norm,0.7094,0.87,0.1491,0.0382,51.00,0.0,cut
475,475,sn94D,Ia-norm,0.6904,0.83,0.1531,0.0455,40.30,0.0,cut
476,476,sn90N,Ia-norm,0.6471,0.79,0.0977,0.0485,245.20,0.0,cut
477,477,sn90N,Ia-norm,0.6363,0.79,0.0966,0.0351,309.80,0.0,cut


In [55]:
file = 'spectra/2023gps/SN2023gps_2023-04-24_00-00-00_NOT_ALFOSC_TNS_snid.output'
with open(file) as f:
    skiprows = 0
    for i, line in enumerate(f.readlines()):   
        if  line.startswith('#no. sn type'):
            break
        skiprows += 1
        
df = pd.read_csv(file, skiprows=skiprows, nrows=5, delim_whitespace=True)

In [56]:
df

,#no.,sn,type,lap,rlap,z,zerr,age,age_flag,grade
0,1,sn05cf,Ia-norm,0.8597,10.08,0.0287,0.0072,-10.94,0,good
1,2,sn02bo,Ia-norm,0.6918,9.17,0.0216,0.0071,-1.50,0,good
2,3,sn02bo,Ia-norm,0.6877,8.66,0.0238,0.0075,-3.40,0,good
3,4,sn95D,Ia-norm,0.6322,8.42,0.0119,0.0071,8.10,0,good
4,5,sn02bo,Ia-norm,0.6918,8.18,0.0352,0.0084,-12.30,0,good
